In [1]:
import os
from dotenv import load_dotenv

load_dotenv('/home/joaovfalves/Documentos/python_rag_langchains/scripts/.env')

os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [2]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema import Document

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

E0000 00:00:1758829699.032102   65828 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1758829699.036124   65828 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [4]:
embeddings.embed_query("Politica de home office da empresa")

[0.060619182884693146,
 -0.01542010810226202,
 -0.06212041899561882,
 0.023809945210814476,
 0.06316284090280533,
 0.029766583815217018,
 0.0018155495636165142,
 -0.032037533819675446,
 -0.011676487512886524,
 0.06364011019468307,
 -0.012819056399166584,
 -0.0016329706413671374,
 -0.040820907801389694,
 -0.00236334465444088,
 0.013701044954359531,
 -0.024350205436348915,
 0.022137722000479698,
 0.02293112315237522,
 0.007548737805336714,
 -0.04146314784884453,
 0.018832029774785042,
 -0.0017550202319398522,
 -0.019857680425047874,
 0.011796340346336365,
 0.027643002569675446,
 -0.01334823202341795,
 0.015560275875031948,
 -0.05682041123509407,
 -0.03708682581782341,
 0.02047661878168583,
 -0.03246597573161125,
 0.0333460196852684,
 -0.06199903413653374,
 0.0201058778911829,
 -0.0003085284843109548,
 -0.012233582325279713,
 -0.029823532328009605,
 -0.014842462725937366,
 0.002058691345155239,
 -0.00016367222997359931,
 0.029233988374471664,
 -0.012414014898240566,
 -0.049305208027362823

In [5]:
documentos_empresa = [
    Document(
        page_content="Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.",
        metadata={"tipo": "política", "departamento": "RH", "ano": 2024, "id_doc": "doc001"}
    ),
    Document(
        page_content="Processo de reembolso de despesas: Envie a nota fiscal pelo portal financeiro. O reembolso ocorre em até 5 dias úteis.",
        metadata={"tipo": "processo", "departamento": "Financeiro", "ano": 2023, "id_doc": "doc002"}
    ),
    Document(
        page_content="Guia de TI: Para configurar a VPN, acesse vpn.nossaempresa.com e siga as instruções para seu sistema operacional.",
        metadata={"tipo": "tutorial", "departamento": "TI", "ano": 2024, "id_doc": "doc003"}
    ),
    Document(
        page_content="Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.",
        metadata={"tipo": "política", "departamento": "RH", "ano": 2022, "id_doc": "doc004"}
    )
]

In [6]:
from langchain_community.vectorstores import FAISS
import faiss

d = 768
index_hsnw2 = faiss.IndexHNSWFlat(d, 32)



In [7]:
faiss_db = FAISS.from_documents(documentos_empresa, embeddings)

pergunta = "Como peço as minhas ferias"

resultados = faiss_db.similarity_search(pergunta, k=2)

In [8]:
print(f"\n Pergunta '{pergunta}'")
print("/n Documentos mais relevantes (FAISS)")
for doc in resultados:
    print(f"- {doc.page_content}")
    print(f" (Metadados: {doc.metadata})")


 Pergunta 'Como peço as minhas ferias'
/n Documentos mais relevantes (FAISS)
- Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.
 (Metadados: {'tipo': 'política', 'departamento': 'RH', 'ano': 2024, 'id_doc': 'doc001'})
- Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.
 (Metadados: {'tipo': 'política', 'departamento': 'RH', 'ano': 2022, 'id_doc': 'doc004'})


In [9]:
from langchain_community.vectorstores import Chroma

chroma_db = Chroma.from_documents(
    documents=documentos_empresa,
    embedding=embeddings
)

In [11]:
resultados = chroma_db.similarity_search(pergunta, k=2)
print(f"\n Pergunta '{pergunta}'")
print("\n Documentos mais relevantes (Chroma)")
for doc in resultados:
    print(f"- {doc.page_content}")
    print(f" (Metadados: {doc.metadata})")


 Pergunta 'Como peço as minhas ferias'

 Documentos mais relevantes (Chroma)
- Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.
 (Metadados: {'ano': 2024, 'tipo': 'política', 'id_doc': 'doc001', 'departamento': 'RH'})
- Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.
 (Metadados: {'id_doc': 'doc004', 'ano': 2022, 'tipo': 'política', 'departamento': 'RH'})


In [14]:
pergunta_rh = "Quais são as regras da empresa"

resultados_filtrados = chroma_db.similarity_search(pergunta_rh, k=2, filter={"$and": [{"departamento": "RH"}, {"tipo": "política"}]})

In [15]:
print(f"\n Pergunta '{pergunta}' com filtro para politicas de RH")
print("\n Documentos mais relevantes e filtrados (Chroma)")
for doc in resultados_filtrados:
    print(f"- {doc.page_content}")
    print(f" (Departamentos: {doc.metadata['departamento']}, Tipo: {doc.metadata['tipo']})")


 Pergunta 'Como peço as minhas ferias' com filtro para politicas de RH

 Documentos mais relevantes e filtrados (Chroma)
- Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.
 (Departamentos: RH, Tipo: política)
- Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.
 (Departamentos: RH, Tipo: política)


In [17]:
from langchain_pinecone import Pinecone
from pinecone import Pinecone as PineconeClient
from pinecone import ServerlessSpec

os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')

pinecone_client = PineconeClient(api_key=os.environ["PINECONE_API_KEY"])

spec = ServerlessSpec(cloud='aws', region='us-east-1')

In [18]:
# Verificando se o índice existe. Se não, ele será criado.
# Nome do seu índice no Pinecone
index_name = "langchain-rag"
if index_name not in pinecone_client.list_indexes().names():
    pinecone_client.create_index(
        name=index_name,
        dimension=d,
        metric="cosine",
        spec=spec
    )
    print(f"Índice '{index_name}' criado no Pinecone.")

    pinecone_db = Pinecone.from_documents(
        documentos_empresa,
        embeddings,
        index_name=index_name
    )

    print(f"Documentos adicionados ao índice '{index_name}'")

else:
    print(f"Conectando ao índice existente '{index_name}'.")
    # Se o índice já existe, apenas carregamos
    pinecone_db = Pinecone.from_existing_index(
        index_name,
        embeddings
    )

Índice 'langchain-rag' criado no Pinecone.
Documentos adicionados ao índice 'langchain-rag'


In [19]:
if pinecone_db:
  # Busca por similaridade
  pergunta_ti = "Como configuro a VPN?"
  resultados_pinecone = pinecone_db.similarity_search(pergunta_ti, k=2)
  print(f"\n Pergunta: {pergunta_ti}")
  print(f"\n Documentos mais relevantes (Pinecone):")
  for doc in resultados_pinecone:
    print(f"- {doc.page_content}")
    print(f"(Metadados: {doc.metadata})")

  # Busca com filtro (Pinecone também suporta)
  resultados_pinecone_filtrados = pinecone_db.similarity_search("informações sobre regras", 
                                                                 k=2,
                                                                 filter={"tipo": "politica"})

  print(f"\n Pergunta: 'Informações sobre regras' com filtro para tipo='politica'")
  print(f"\n Documentos relevantes e filtrados (Pinecone):")
  for doc in resultados_pinecone_filtrados:
    print(f"- {doc.page_content}")
    print(f"(Tipo: {doc.metadata['tipo']})")


 Pergunta: Como configuro a VPN?

 Documentos mais relevantes (Pinecone):
- Guia de TI: Para configurar a VPN, acesse vpn.nossaempresa.com e siga as instruções para seu sistema operacional.
(Metadados: {'ano': 2024.0, 'departamento': 'TI', 'id_doc': 'doc003', 'tipo': 'tutorial'})
- Processo de reembolso de despesas: Envie a nota fiscal pelo portal financeiro. O reembolso ocorre em até 5 dias úteis.
(Metadados: {'ano': 2023.0, 'departamento': 'Financeiro', 'id_doc': 'doc002', 'tipo': 'processo'})

 Pergunta: 'Informações sobre regras' com filtro para tipo='politica'

 Documentos relevantes e filtrados (Pinecone):
